<a href="https://colab.research.google.com/github/KaifAhmad1/AB-Testing/blob/master/Metaforms_Assingment_AI_Engineer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Assignemnt:**  To create a prompt or a model that takes a goal as an input from users (product
 managers, user researchers, market researchers, SMB owners) and returns a list of questions
 for a survey that will help reach the goal.

In [9]:
!pip --quiet install transformers accelerate bitsandbytes langchain

In [10]:
from huggingface_hub import notebook_login
notebook_login()

In [12]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import BitsAndBytesConfig
import torch

model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
quantization_config = BitsAndBytesConfig(load_in_4bit=True,bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16)

model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", quantization_config=quantization_config)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [13]:
from transformers import pipeline
READER_LLM = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    do_sample=True,
    temperature=0.2,
    repetition_penalty=1.1,
    return_full_text=False,
    max_new_tokens=500,
)

In [14]:
from langchain import PromptTemplate, LLMChain

In [22]:
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate

# Define the few-shot examples
examples = [
    {
        "goal": "Understand customer satisfaction with our recent software update.",
        "survey_questions": """
1. On a scale of 1 to 5, how satisfied are you with the new features introduced in the latest software update?
2. What specific features did you find most useful in the recent update?
3. Were there any issues or bugs you encountered after installing the update?
4. How easy or difficult was it to navigate the new user interface changes?
5. Overall, how would you rate your experience with the latest software update on a scale of 1 to 10?
""",
    },
    {
        "goal": "Gather feedback on a new product concept for a smart home device.",
        "survey_questions": """
1. How interested would you be in purchasing a smart home device that can automate various household tasks?
2. What features would you expect from such a smart home device?
3. What are your primary concerns or hesitations regarding the use of a smart home device?
4. How much would you be willing to pay for a high-quality smart home device with advanced features?
5. Would you prefer a voice-controlled or app-controlled smart home device? Why?
""",
    },
    {
        "goal": "Assess the market demand for a new line of eco-friendly cleaning products.",
        "survey_questions": """
1. How important is it to you to use environmentally friendly cleaning products in your household?
2. What factors do you consider when choosing eco-friendly cleaning products (e.g., ingredients, packaging, price)?
3. What specific types of eco-friendly cleaning products would you be interested in purchasing?
4. How much more would you be willing to pay for eco-friendly cleaning products compared to conventional ones?
5. Where do you typically purchase cleaning products (e.g., grocery stores, online retailers, specialty stores)?
""",
    },
]

In [18]:
# Define the prompt template
prompt_template = PromptTemplate(
    template="""
You are a survey question generator. Based on the provided goal, your task is to generate a list of 5-10 survey questions that can help achieve the goal.

Goal: {goal}

Survey Questions:
{survey_questions}
""",
    input_variables=["goal", "survey_questions"],
)

In [23]:
# Create the FewShotPromptTemplate
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=prompt_template,
    prefix="Here are some examples of survey questions generated for different goals:",
    suffix="Based on the provided goal, generate a list of 5-10 survey questions.",
    input_variables=["goal"],
    example_separator="\n\n",
)

In [27]:
# Generate survey questions based on the input goal
goal = "Understand customer satisfaction with our customer support services."
formatted_prompt = prompt.format(goal=goal, survey_questions="")
survey_questions = READER_LLM(formatted_prompt)

# Extract the generated text from the output
generated_text = survey_questions[0]['generated_text']
print(generated_text)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
